In [1]:
import datetime
from digitforce.aip.common.utils.spark_helper import SparkClient
import findspark
findspark.init()
# 需要固定位置的import 后添加 # NOQA: E402， 表示忽略模块级别导入不在文件顶部的错误
from pyspark.sql import window as W  # NOQA: E402
from pyspark.sql import functions as F  # NOQA: E402
from pyspark.sql import types as T  # NOQA: E402
from utils import *  # NOQA: E402
from digitforce.aip.common.utils.time_helper import DATE_FORMAT
spark_client = SparkClient.get()
spark = spark_client.get_session()

spark_config: 
{
    "hive_uris": "thrift://172.22.20.137:7004,thrift://172.22.20.110:7004",
    "java_home": "/opt/jdk1.8.0_181-cloudera",
    "kubernetes_namespace": "kubeflow-user-example-com",
    "kubernetes_runtime_image": "digit-force-docker.pkg.coding.net/ai-platform/base-images/spark-k8s-runtime-with-jars:2.4.8",
    "master_uri": "k8s://https://172.22.20.16",
    "spark_home": "/opt/spark-2.4.8-bin-hadoop2.7"
}


23/03/08 02:11:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from sample_select import *
dixiao_before_days = 3
dixiao_after_days = 3
right_zc_threshold = 100
avg_zc_threshold = 100
event_tag = 'login'
sample_table_name = start_sample_selection(
        dixiao_before_days=dixiao_before_days,
        dixiao_after_days=dixiao_after_days,
        right_zc_threshold=right_zc_threshold,
        avg_zc_threshold=avg_zc_threshold,
        event_tag=event_tag,
    )

333333333333333


23/03/08 02:11:40 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.


In [ ]:
dixiao_before_days = 10
dixiao_after_days = 10
right_zc_threshold=10000
avg_zc_threshold=10000
feature_days = 10
event_tag = 'login'
# dixiao_before_days = max(dixiao_act_before_days,dixiao_zc_before_days)
# # dixiao_after_days = dixiao_after_days
# # right_zc_threshold=10000
# # avg_zc_threshold=10000
# feature_days = 10

In [ ]:
DATE_FORMAT = "%Y%m%d"
user_table = "zq_standard.dm_cust_label_base_attributes_df"
app_table = "zq_standard.dm_cust_traf_behv_aggregate_df"
zj_table = "zq_standard.dm_cust_capital_flow_aggregate_df"
jy_table = "zq_standard.dm_cust_subs_redm_event_aggregate_df"
zc_table = "zq_standard.dm_cust_ast_redm_event_df"
user_view = user_table[user_table.find(".")+1:]
app_view = app_table[app_table.find(".")+1:]
zj_view = zj_table[zj_table.find(".")+1:]
jy_view = jy_table[jy_table.find(".")+1:]
zc_view = zc_table[zc_table.find(".")+1:]


# 1.获取关键时间点
window_test_days = 3
window_train_days = 5
now = datetime.datetime.now()
dixiao_end_date = now - datetime.timedelta(days=2)  # 低效户结束日期
end_date = dixiao_end_date - \
    datetime.timedelta(days=dixiao_after_days)  # 低效户结束日期
mid_date = end_date - datetime.timedelta(days=window_test_days)
start_date = mid_date - datetime.timedelta(days=window_train_days)
dixiao_start_date = start_date - datetime.timedelta(
    days=dixiao_before_days
)  # 低效户开始日期
feature_date = dixiao_start_date - \
    datetime.timedelta(days=feature_days)  # 特征数据最早日期

now = now.strftime(DATE_FORMAT)
dixiao_end_date = dixiao_end_date.strftime(DATE_FORMAT)
end_date = end_date.strftime(DATE_FORMAT)
mid_date = mid_date.strftime(DATE_FORMAT)
start_date = start_date.strftime(DATE_FORMAT)
dixiao_start_date = dixiao_start_date.strftime(DATE_FORMAT)
feature_date = feature_date.strftime(DATE_FORMAT)
# 2. 特征预处理
spark_client.get_starrocks_table_df(
    user_table).createOrReplaceTempView(user_view)
spark_client.get_starrocks_table_df(
    app_table).createOrReplaceTempView(app_view)
spark_client.get_starrocks_table_df(
    zj_table).createOrReplaceTempView(zj_view)
spark_client.get_starrocks_table_df(
    jy_table).createOrReplaceTempView(jy_view)
spark_client.get_starrocks_table_df(
    zc_table).createOrReplaceTempView(zc_view)

In [ ]:
from sample_select import *
# 非交易日列表
noexchangedate_list = noexchange_days(
    start_date=datetime.datetime.strptime(
        dixiao_start_date, '%Y%m%d').strftime('%Y-%m-%d'),
    end_date=datetime.datetime.strptime(
        dixiao_end_date, '%Y%m%d').strftime('%Y-%m-%d'),
)
# 资产数据
zzc_sample_rdd = get_zc_sample(
    spark,
    zc='total_ast',
    zc_view=zc_view,
    dixiao_start_date=dixiao_start_date,
    dixiao_end_date=dixiao_end_date,
    start_date=start_date,
    end_date=end_date,
    dixiao_before_days=dixiao_before_days,
    dixiao_after_days=dixiao_after_days,
    noexchangedate_list=noexchangedate_list
)
zzc_sample = (
    zzc_sample_rdd.toDF(
        ["cust_code", "right_zzc", "past_avg_zzc", "future_max_zzc", "dt"])
)
zc_sample_df = zzc_sample

In [ ]:
zc_sample_df.select('dt').distinct().orderBy(F.desc('dt')).show()

In [ ]:
from utils import *
login_sample = (
    spark.sql(
        f"""
            select cust_code,is_login,replace(dt,'-','') as dt
            from {app_view} 
            where replace(dt,'-','') between '{dixiao_start_date}' and '{dixiao_end_date}'
            """
    )
    .rdd
    .map(lambda x: (x[0], [(x[2], int(x[1]))]))
    .reduceByKey(lambda a, b: a+b)
    # 每个x[0] 下面按照日期排序
    .map(lambda x: (x[0], sorted(x[1], key=lambda y: int(y[0]), reverse=False)))
    # 得到当前日期，过去登陆天数
    .map(lambda x: (x[0], get_login_days(x[1], start_date, end_date, dixiao_before_days, noexchangedate_list)))
    .flatMapValues(lambda x: x)
    .map(lambda x: (x[0], x[1][1], x[1][0]))
)

login_sample_df = login_sample.toDF(
    ["cust_code", "before_login_days", "dt"])

# 交易数据
exchange_sample = (
    spark.sql(
        f"""
            select cust_code,total_tran_cnt,replace(dt,'-','') as dt
            from {jy_view} 
            where replace(dt,'-','') between '{dixiao_start_date}' and '{dixiao_end_date}'
            """
    )
    .rdd
    .map(lambda x: (x[0], [(x[2], int(x[1]))]))
    # 每个x[0] 下面按照日期排序
    .map(lambda x: (x[0], sorted(x[1], key=lambda y: int(y[0]), reverse=False)))
    .reduceByKey(lambda a, b: a+b)
    # 得到当前日期，过去登陆天数
    .map(lambda x: (x[0], get_exchange_days(x[1], start_date, end_date, dixiao_before_days, noexchangedate_list)))
    .flatMapValues(lambda x: x)
    .map(lambda x: (x[0], x[1][1], x[1][0]))
)

In [ ]:
# import pandas as pd
_schema1 = T.StructType([
    T.StructField('cust_code', T.StringType(),True),
    T.StructField('before_exchange_days',T.LongType(),True),
    T.StructField('dt',T.StringType(),True)
])
exchange_sample_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),_schema1)

In [ ]:
exchange_sample_df.rdd.isEmpty()


In [ ]:

exchange_sample_df = exchange_sample.toDF(["cust_code", "before_exchange_days", "dt"])

In [ ]:

sample_df = (
    zc_sample_df
    .join(login_sample_df, on = ['cust_code','dt'],how='left')
    .join(exchange_sample_df, on = ['cust_code','dt'],how='left')
    .filter(
        F.expr(
            f"""
            right_zzc<{right_zc_threshold} and right_zzc>0 and past_avg_zzc<{avg_zc_threshold}
            """
        )
    ) # 过滤时点资产和平均资产
)
if event_tag == 'login':
    sample_df = sample_df.filter(F.expr(f"before_login_days>0"))
elif event_tag == 'exchange':
    sample_df = sample_df.filter(F.expr(f"before_exchange_days>0"))
    
sample_df = (
    sample_df.withColumn(
        "label",
        F.expr(f"IF(future_max_zzc>{right_zc_threshold},'1','0')")
    )
    .select("cust_code", "label", "dt")
)
sample_df.show()

In [ ]:
def write_hive(spark, inp_df, table_name, partition_col):
    check_table = (
        spark._jsparkSession.catalog().tableExists(table_name)
    )

    if check_table:  # 如果存在该表
        print("table:{} exist......".format(table_name))
        inp_df.write.format("orc").mode("overwrite").insertInto(table_name)

    else:  # 如果不存在
        print("table:{} not exist......".format(table_name))
        inp_df.write.format("orc").mode("overwrite").partitionBy(
            partition_col
        ).saveAsTable(table_name)

In [ ]:
# from sample_select import write_hive
sample_table_name = "algorithm.aip_zq_dixiaohu_custom_label"
write_hive(spark, sample_df, sample_table_name, "dt")

In [ ]:
# 客户号，年龄，性别，城市，省份，教育程度(end_date 的基础信息)
table_user = spark.sql(
    f"""
    select cust_code, age, gender, city_name, province, educational_degree from {user_view} where replace(dt,'-','') = '{end_date}'
    """
)
# 客户号，日期，客户是否登录
table_app = spark.sql(
    f"""
    select cust_code, replace(dt,'-','') as dt, is_login from {app_view} where replace(dt,'-','') between '{feature_date}' and '{end_date}'
    """
)
# 客户号，日期，资金转出金额，资金转入金额，资金转出笔数，资金转入笔数
table_zj = spark.sql(
    f"""
    select cust_code, replace(dt,'-','') as dt, zc_money, zr_money, zc_cnt, zr_cnt 
    from {zj_view} 
    where replace(dt,'-','') between '{feature_date}' and '{end_date}'
    """
)
# 客户号，日期，交易笔数，交易金额，股票笔数，股票金额，基金笔数，基金金额
table_jy = spark.sql(
    f"""
    select cust_code, replace(dt,'-','') as dt, jy_num, jy_rmb, jygp_num, jygp_rmb, jyjj_num, jyjj_rmb 
    from {jy_view} 
    where replace(dt,'-','') between '{feature_date}' and '{end_date}'
    """
)
# 客户号，日期，总资产，总负债，基金资产，股票资产，资金余额，产品资产
table_zc = spark.sql(
    f"""
    select cust_code, replace(dt,'-','') as dt, ast_total, ast_fz, ast_jj, ast_gp, ast_zj, ast_cp 
    from {zc_view} 
    where replace(dt,'-','') between '{feature_date}' and '{end_date}'
    """
)

# 3. 特征融合
data = (
    spark.sql(
        f"""
        SELECT cust_code,label,dt
        FROM {sample_table_name}
        WHERE dt>= {dixiao_start_date} and dt <= {end_date}
        """
    )
    .join(table_user, on=["cust_code"], how="left")
    .join(table_app, on=["cust_code", "dt"], how="left")
    .join(table_zj, on=["cust_code", "dt"], how="left")
    .join(table_jy, on=["cust_code", "dt"], how="left")
    .join(table_zc, on=["cust_code", "dt"], how="left")
)
# TODO: 特征加工

data_train_df = data.filter(F.col("dt") <= mid_date)
data_test_df = data.filter(F.col("dt") > mid_date)

In [ ]:
train_table_name = "algorithm.aip_zq_dixiaohu_custom_feature_train"
test_table_name = "algorithm.aip_zq_dixiaohu_custom_feature_test"
write_hive(spark, data_train_df, train_table_name, "dt")
write_hive(spark, data_test_df, test_table_name, "dt")

In [ ]:
import pandas as pd
from digitforce.aip.common.utils.hdfs_helper import hdfs_client

# 处理分类特征，数值特征，需要丢掉的特征
def featuretype_process(data_init, drop_labels, categorical_feature, float_feature):
    """TODO:自动识别连续特征和离散特征"""
    # Process Feature 1
    data_process = data_init.drop(
        labels=drop_labels, axis=1, errors="ignore"
    )
    data_process[categorical_feature] = (
        data_process[categorical_feature].astype("str").astype("category")
    )
    # data_process[float_feature] = data_process[float_feature].astype('float')
    data_process[float_feature] = data_process[float_feature].apply(
        lambda col: pd.to_numeric(col, errors="coerce"), axis=0
    )
    return data_process

def write_hdfs_path(local_path, hdfs_path):
    if hdfs_client.exists(hdfs_path):
        hdfs_client.delete(hdfs_path)
    hdfs_client.copy_from_local(local_path, hdfs_path)

In [ ]:
learning_rate=0.05,
n_estimators=200,
max_depth=5,
scale_pos_weight=0.5,
is_automl=False,
model_and_metrics_data_hdfs_path=None,

In [ ]:
import random
import lightgbm as lgb
import joblib
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    log_loss,
)


df_train = (
    spark
    .sql(
        f"""
        select * from {train_table_name} limit 1000000
        """
    )
    .toPandas()
)

print("&&&&&&&&&&&&&&&&&&&&&&&", len(df_train))
df_test = (
    spark
    .sql(
        f"""
        select * from {test_table_name}
        """
    )
    .toPandas()
)

print("#######################", len(df_test))
# 连续特征，离散特征，丢弃特征等的处理
drop_features = []
categorical_features = [
    "gender",
    "city_name",
    "province",
    "educational_degree",
    "is_login",
]
float_features = [
    "age",
    "jy_num",
    "jy_rmb",
    "jygp_num",
    "jygp_rmb",
    "jyjj_num",
    "jyjj_rmb",
    "ast_total",
    "ast_fz",
    "ast_jj",
    "ast_gp",
    "ast_zj",
    "ast_cp",
    "zc_money",
    "zr_money",
    "zc_cnt",
    "zr_cnt",
]
df_train = featuretype_process(
    data_init=df_train,
    drop_labels=drop_features,
    categorical_feature=categorical_features,
    float_feature=float_features,
)
df_test = featuretype_process(
    data_init=df_test,
    drop_labels=drop_features,
    categorical_feature=categorical_features,
    float_feature=float_features,
)

x_train = df_train.drop(columns=["cust_code", "label", "dt"], axis=1)
y_train = df_train["label"]

x_test = df_test.drop(columns=["cust_code", "label", "dt"], axis=1)
y_test = df_test["label"]

# TODO：mock数据临时修改label
random.seed(1234)
y_train = y_train.map(lambda x: random.randint(0, 1))
y_test = y_test.map(lambda x: random.randint(0, 1))


In [ ]:

# 模型训练
model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    objective="binary",
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    max_depth=max_depth,
    scale_pos_weight=scale_pos_weight,
    eval_metric="logloss",
)

model.fit(x_train, y_train, verbose=True)


# 测试集打分&效果评估
y_pred = model.predict(x_test)
y_pred_score = [x[1] for x in model.predict_proba(x_test)]

def getRates(y_test, y_pred, y_pred_score):
    s_acc = accuracy_score(y_test, y_pred)
    s_auc = roc_auc_score(y_test, y_pred_score)
    s_pre = precision_score(y_test, y_pred)
    s_rec = recall_score(y_test, y_pred)
    s_f1 = f1_score(y_test, y_pred)
    s_loss = log_loss(y_test, y_pred_score)
    return [s_acc, s_auc, s_pre, s_rec, s_f1, s_loss]

all_score = getRates(y_test, y_pred, y_pred_score)
print("test-logloss={:.4f}, test-auc={:.4f}".format(all_score[5], all_score[1]))

if not is_automl: # automl 默认值这里给False
    local_file_path = "{}_aip_zq_dixiaohu.model".format(today)
    joblib.dump(model, local_file_path)
    hdfs_path1 = "/user/ai/aip/zq/dixiaohu/model/{}.model".format(today)
    hdfs_path2 = "/user/ai/aip/zq/dixiaohu/model/latest.model"
    write_hdfs_path(local_file_path, hdfs_path1)
    write_hdfs_path(local_file_path, hdfs_path2)